# 제품 이상여부 판별 프로젝트


## hard voting (csv 파일 이용)

모델을 통해 예측해 낸 정답지(csv) 파일의 결과값을 불러와 voting 하는 방식  
이때의 voting은 hard방식만 가능(예측확률은 저장되어있지 않음으로 soft 보팅은 불가능)  

In [1]:
import pandas as pd
import numpy as np
from collections import Counter

def read_submission_files(file_paths):
    """
    제출 파일을 읽어와서 예측 결과를 반환합니다.

    Parameters:
    file_paths (list of str): 제출 파일 경로 리스트

    Returns:
    list of np.array: 각 제출 파일의 예측 결과 리스트
    """
    predictions = []
    for file_path in file_paths:
        df = pd.read_csv(file_path)
        preds = df['target'].apply(lambda x: 1 if x == 'AbNormal' else 0).values
        predictions.append(preds)
    return predictions

def hard_voting(preds):
    """
    하드 보팅을 사용하여 최종 예측을 수행합니다.

    Parameters:
    preds (list of np.array): 각 모델의 예측 배열 리스트

    Returns:
    np.array: 최종 예측 결과
    """
    preds = np.array(preds)
    
    # 각 샘플의 예측 결과를 문자열로 변환하여 리스트에 저장
    sample_predictions = [''.join(map(str, x)) for x in preds.T]
    
    # 각 예측 결과의 빈도수를 계산
    prediction_counts = Counter(sample_predictions)
    
    # 빈도수 출력
    for pred, count in prediction_counts.items():
        print(f"Prediction {pred}: {count} times")
    
    # 하드 보팅을 통해 최종 예측을 계산
    final_predictions = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=preds)
    return final_predictions

submission_file 에서 csv 파일을 불러와 hard voting 하는 알고리즘  

In [2]:
import pandas as pd
from collections import Counter

# 공통 경로
common_path = "submission_file/"

# 제출 파일 이름 리스트
file_names = [
    "1012_kcbert_base.csv"
    , "1012_roberta_base(batch_32_max256).csv"
    , "1012_kcelectra_base_200.csv"

    , "1012_electra_kor_base.csv"
    , "1012_KoBigBird-base_lr.csv"

    # 파일 추가 가능  <----- 파일 필요시 추가하세요!!
]

# 경로를 추가하는 함수
def add_common_path(file_names, common_path):
    return [common_path + file_name for file_name in file_names]

# 제출 파일에서 예측 결과 읽어오기
def read_submission_files(file_paths):
    predictions = []
    for file_path in file_paths:
        df = pd.read_csv(file_path)
        if 'ID' not in df.columns or '분류' not in df.columns:
            raise KeyError(f"'ID' 또는 '분류' 열이 {file_path} 파일에 없습니다. 열 이름: {df.columns.tolist()}")
        predictions.append(df[['ID', '분류']])
    return predictions

# 하드 보팅 함수
def hard_voting(predictions):
    final_predictions = {}
    id_list = predictions[0]['ID'].tolist()
    
    for id in id_list:
        votes = [df[df['ID'] == id]['분류'].values[0] for df in predictions]
        vote_counts = Counter(votes)
        if vote_counts.most_common(1)[0][1] > 1:
            final_predictions[id] = vote_counts.most_common(1)[0][0]
        else:
            final_predictions[id] = votes[0]
    
    return final_predictions

# 경로가 추가된 파일 리스트
file_paths = add_common_path(file_names, common_path)

# 제출 파일에서 예측 결과 읽어오기
predictions = read_submission_files(file_paths)

# 하드 보팅 결과
final_predictions_hard = hard_voting(predictions)

# 결과를 새로운 제출 파일로 저장할 파일 이름
output_file_name = "bert_voting_1012_5개모델.csv"          # <----- 파일 이름을 변경하세요!!

# 결과를 새로운 제출 파일로 저장
df_sub = pd.read_csv(file_paths[0])
df_sub['분류'] = df_sub['ID'].apply(lambda x: final_predictions_hard[x])
df_sub.to_csv(output_file_name, index=False)

print(f"최종 제출 파일이 '{output_file_name}'로 저장되었습니다.")

최종 제출 파일이 'bert_voting_1012_5개모델.csv'로 저장되었습니다.


In [3]:
df_sub['분류'].value_counts()

지역               12150
경제:부동산            1491
사회:사건_사고          1140
경제:반도체             957
사회:사회일반            514
사회:교육_시험           453
정치:국회_정당           434
사회:의료_건강           402
스포츠:올림픽_아시안게임      367
경제:취업_창업           355
스포츠:골프             289
경제:산업_기업           285
경제:자동차             277
정치:선거              271
문화:전시_공연           266
경제:유통              247
사회:장애인             247
IT_과학:모바일          222
사회:여성              193
경제:경제일반            193
사회:노동_복지           184
경제:서비스_쇼핑          181
경제:무역              162
문화:방송_연예           156
스포츠:축구             143
경제:금융_재테크          139
사회:환경              125
정치:행정_자치           122
정치:청와대             119
국제                 108
IT_과학:인터넷_SNS       93
문화:미술_건축            88
문화:학술_문화재           87
문화:출판               86
IT_과학:과학            82
IT_과학:IT_과학일반       80
경제:자원               72
문화:문화일반             70
문화:요리_여행            70
문화:종교               67
사회:날씨               56
정치:정치일반             54
IT_과학:콘텐츠           53
스포츠:농구_배구  

In [4]:
df_sub.head(10)

,ID,분류
0,TEST_00000,사회:사회일반
1,TEST_00001,사회:사회일반
2,TEST_00002,정치:행정_자치
3,TEST_00003,경제:취업_창업
4,TEST_00004,지역
5,TEST_00005,경제:반도체
6,TEST_00006,정치:정치일반
7,TEST_00007,국제
8,TEST_00008,사회:사건_사고
9,TEST_00009,경제:자원
